## 2.2 Prédiction du temps de retard juste après le départ


Pour cette approche du problème, on cherche à prévoir le temps de retard prévu pour les trains qu’on sait être en retard au départ. Cette prédiction peut être utile à plusieurs égards :
- Pour les utilisateurs d’abord, à qui on peut prédire le retard auquel ils peuvent s'attendre à l'arrivée lorsqu’ils apprennent que leur train est en retard au départ. La SNCF cherche d’ailleurs déjà à partager cette information à ses clients à travers l’application SNCF ou sur les différents affichages en gare et dans les trains. 
- Cela permet aussi à la SNCF de prendre des mesures efficaces s'il y a des correspondances et de donner une première information aussi précise que possible à ses clients. 

Un train en retard au départ n’est en général plus prioritaire et ne pourra être mis en circulation qu’au prochain créneau libre. Il se peut qu’il soit tout de même priorisé si son retard perturbe de nombreux autres trajets (dans le cas de correspondances notamment). Deux cas de figure sont donc envisageables lorsqu’un train est en retard au départ : 
- Le train peut encore être priorisé pour compenser son retard et il a de chances d’arriver à l’heure
- Le train ne peut plus être priorisé et on est sûr qu’il sera en retard à l’arrivée

Dans le premier cas, nous chercherons à prédire retard_moyen_tous_trains_arrivee, dans le deuxième cas nous chercherons à prédire retard_moyen_arrivee (le train ne peut plus arriver à l’heure, il fait déjà partie de la catégorie des trains en retard à l’arrivée). La démarche suivie ci-après est exactement la même dans les deux cas à cette exception près. 


Avant propos : 

Une méthode de discrétisation des retards a été expérimentée sans grand succès. Les catégories utilisées étaient les suivantes : 0-10, 10-20, 20-30, 30-45, 45-60, >60. Malheureusement, l'accuracy obtenue était de 60% au mieux avec les différents modèles testés, ce qui n'était pas convaincant. Les modèles de regression ont donc été retenus pour la suite du travail. 

In [ ]:
#Cellule d'importation

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
#Choix du cas considéré. Assignez 'retard_moyen_tous_trains arrivee' à column pour le cas 1 (train priorisable), 'retard_moyen_arrivee' sinon (train non priorisable).

column = 'retard_moyen_arrivee'

Nous reprenons les mêmes hypothèses et faisons les mêmes traintements que dans la partie 2.1 pour commencer. Nous ajoutons en plus des données déjà utilisées précédemment les données suivantes : 
- Le nombre de trains en retard au départ dans le mois d’intérêt
- Le retard moyen des trains en retard au départ dans le mois d’intérêt.

Pour avoir ces données à n'importe quel moment du mois, il faudrait un modèle qui prédit le nombre de retards au départ et leurs moyenne à partir du nombre de retards et leur moyenne jusqu’à la date considérée. Ce problème n’a pas été traité ici mais nous partons du principe qu’il existe et qu’il est efficace. 


En plus de ce qui a été fait dans la partie 2.1, nous effectuons également un one-hot-encoding sur la présence ou non de commentaires concernant les retards le mois d’avant. En effet, ces commentaires indiquent des causes extraordinaires ayant influencé les retards. Ceci pourrait influencer les retards du mois d’après. 
On ajoute également les pourcentages de chacune des causes des 4 mois précédant le mois d’intérêt ainsi que la moyenne sur tous les mois précédents. Certaines causes impliquent des mesures chronophages qui peuvent influer sur les retards des mois futurs. 


In [ ]:

data = pd.read_csv('../dataset/regularite-mensuelle-tgv-aqst.csv')
df = pd.DataFrame(data)

#On ajoute un colonne année 
annee = []
mois = []
for i in df['date'].values :
    annee.append(int(i[:4]))
    mois.append(int(i[5:]))

df['annee'] = annee
df['mois'] = mois

#On supprime la colonne date

df = df.drop(['date'], axis = 1)


#On transforme la colonne commentaire, si un commentaire est présent, on met un 1, sinon un 0. Les commentaires signifie en général des éléments extraordinaire qui peuvent augmenter les retards.
def comm(com):
    if pd.isna(com): 
        return 0
    else :
        return 1
    
#On fait la transformation
df.loc[:, 'commentaires_retard_arrivee'] = df['commentaires_retard_arrivee'].apply(comm)


#On ajoute une colonne "retards_moy_arrivee_trajet_passés"

def moyenne_donnee_passee(df, column):
    df[column +"_passe"] = 0.0
    for i,row in df.iterrows():
        df.loc[i, column + "_passe" ] = df[(df['annee'] <= row['annee']) & (df['gare_depart']== row['gare_depart']) & (df['gare_arrivee']== row['gare_arrivee']) & (df['mois'] <= row['mois'])][column].mean()
    return df




#On crée la fonction qui permet d'ajouter les valeurs des retards des mois précédents

def add_data_previous_months(df, n_months, column):
    past_month_cols = [f"{column}_mois_n-{i}" for i in range(1, n_months+1)]
    for col in past_month_cols:
        df[col] = 0.0
    for i,row in df.iterrows():
        annee, mois = row["annee"], row["mois"]
        gare_depart, gare_arrivee = row["gare_depart"], row["gare_arrivee"]
        for j, col_name in enumerate(past_month_cols):
            n_mois_a_soustraire = j+1
            date_mois_prec = (annee, mois-n_mois_a_soustraire) if mois>=n_mois_a_soustraire+1 else ((annee-1, 12+mois-n_mois_a_soustraire) if annee>=2019 else (annee, mois))
            series = df.loc[(df.gare_depart == gare_depart) & (df.gare_arrivee == gare_arrivee) & (df.annee == date_mois_prec[0]) & (df.mois == date_mois_prec[1]), column]
            
            if not series.empty:
                value = series.values[0]
                
                df.loc[i, col_name] = value
            else:
                
                df.loc[i, col_name] = row[column + "_passe"]
    return df



# Méthode de Tuckey pour les outliers

def smooth_outliers(val, outlier_step, Q1, Q3):
    if val < Q1-outlier_step:
        val = Q1-outlier_step
    elif val > Q3+outlier_step:
        val = Q3+outlier_step
    return val

def clean_outliers(df, outlier_cols):
    # df = input_df.copy(deep=True)
    for col_name in outlier_cols:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col_name], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col_name],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        # outlier step
        outlier_step = 1.5 * IQR
        df[col_name] = df[col_name].apply(lambda x: smooth_outliers(x, outlier_step, Q1, Q3))
    return df



In [ ]:
#gestion des outliers
outlier_cols = ["duree_moyenne", "nb_train_prevu", column]
df = clean_outliers(df, outlier_cols)

In [ ]:
#Ajout des moyennes des mois passés


df = moyenne_donnee_passee(df, column )
df = moyenne_donnee_passee(df, "prct_cause_externe" )
df = moyenne_donnee_passee(df, "prct_cause_infra" )
df = moyenne_donnee_passee(df, "prct_cause_gestion_trafic" )
df = moyenne_donnee_passee(df, "prct_cause_materiel_roulant" )
df = moyenne_donnee_passee(df, "prct_cause_gestion_gare" )
df = moyenne_donnee_passee(df, "prct_cause_prise_en_charge_voyageurs" )

In [ ]:
#On ajoute les données des 4 mois précédents et on affiche le dataframe pour les retards à l'arrivée et les pourcentages de causes.

df = add_data_previous_months(df,4 ,column )
df = add_data_previous_months(df,4 , "prct_cause_externe" )
df = add_data_previous_months(df,4 , "prct_cause_infra" )
df = add_data_previous_months(df,4 , "prct_cause_gestion_trafic" )
df = add_data_previous_months(df,4 , "prct_cause_materiel_roulant" )
df = add_data_previous_months(df,4 , "prct_cause_gestion_gare" )
df = add_data_previous_months(df,4 , "prct_cause_prise_en_charge_voyageurs" )



In [ ]:
#On enlève les données des quatre premiers mois du dataset où les valeurs des mois précédents n'existe pas. Ces lignes pourraient fausser la donnée.

df = df.drop(df[(df['mois'] == 1) & (df['annee'] == 2018)].index)
df = df.drop(df[(df['mois'] == 2) & (df['annee'] == 2018)].index)
df = df.drop(df[(df['mois'] == 3) & (df['annee'] == 2018)].index)
df = df.drop(df[(df['mois'] == 4) & (df['annee'] == 2018)].index)

In [ ]:
#Observons les correlations entre les features et le donnée que nous cherchons à prédire.


data_f = df[['annee', 'mois','service', 'gare_depart', 'gare_arrivee', 'duree_moyenne', 'nb_train_prevu','nb_train_depart_retard', 
            'retard_moyen_depart', column +'_mois_n-1', column +'_mois_n-2', column +'_mois_n-3', column +'_mois_n-4',
             'commentaires_retard_arrivee', column +'', column +"_passe",
                'prct_cause_externe_passe','prct_cause_externe_mois_n-1', 'prct_cause_externe_mois_n-2', 'prct_cause_externe_mois_n-3', 'prct_cause_externe_mois_n-4',
                'prct_cause_infra_passe','prct_cause_infra_mois_n-1', 'prct_cause_infra_mois_n-2', 'prct_cause_infra_mois_n-3', 'prct_cause_infra_mois_n-4',
                'prct_cause_gestion_trafic_passe','prct_cause_gestion_trafic_mois_n-1', 'prct_cause_gestion_trafic_mois_n-2', 'prct_cause_gestion_trafic_mois_n-3', 'prct_cause_gestion_trafic_mois_n-4',
                'prct_cause_materiel_roulant_passe','prct_cause_materiel_roulant_mois_n-1', 'prct_cause_materiel_roulant_mois_n-2', 'prct_cause_materiel_roulant_mois_n-3', 'prct_cause_materiel_roulant_mois_n-4',
                 'prct_cause_gestion_gare_passe','prct_cause_gestion_gare_mois_n-1', 'prct_cause_gestion_gare_mois_n-2', 'prct_cause_gestion_gare_mois_n-3', 'prct_cause_gestion_gare_mois_n-4',
                  'prct_cause_prise_en_charge_voyageurs_passe','prct_cause_prise_en_charge_voyageurs_mois_n-1', 'prct_cause_prise_en_charge_voyageurs_mois_n-2', 'prct_cause_prise_en_charge_voyageurs_mois_n-3', 'prct_cause_prise_en_charge_voyageurs_mois_n-4']]


plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(data_f.corr()[[column]].sort_values(by=column, ascending=False), vmin=-1, vmax=1, annot=True, cmap='coolwarm')
heatmap.set_title('Features Correlating with {}'.format(column), fontdict={'fontsize':18}, pad=16)

On a réussi à obtenir quelques niveaux de corrélation intéressants et certaines valeurs confirment nos hypothèses. Etant donne que ces pourcentages sont liés (leur somme vaut 1), on peut en déduire que les causes externes ont un fort impact sur les retards alors que la gestion gare et la gestion trafic beaucoup moins. Lorsque ces dernières causes ont un pourcentage élevé, les retards sont plus faibles (corrélation négative). On a également l'impression que les mêmes lignes sont souvent touchées par les mêmes causes car un ligne qui a eu dans le passé des gros pourcentage de causes externes à de fortes chances d'avoir de gros retards dans le futur. 

In [ ]:

#On sépare X et y

def transfo_colonnes(a, column):

     

     X = a[['annee', 'mois','service', 'gare_depart', 'gare_arrivee', 'duree_moyenne', 'nb_train_prevu','nb_train_depart_retard', 
            'retard_moyen_depart', column +'_mois_n-1', column +'_mois_n-2', column +'_mois_n-3', column +'_mois_n-4',
             'commentaires_retard_arrivee', column +"_passe",
                'prct_cause_externe_passe','prct_cause_externe_mois_n-1', 'prct_cause_externe_mois_n-2', 'prct_cause_externe_mois_n-3', 'prct_cause_externe_mois_n-4',
                'prct_cause_infra_passe','prct_cause_infra_mois_n-1', 'prct_cause_infra_mois_n-2', 'prct_cause_infra_mois_n-3', 'prct_cause_infra_mois_n-4',
                'prct_cause_gestion_trafic_passe','prct_cause_gestion_trafic_mois_n-1', 'prct_cause_gestion_trafic_mois_n-2', 'prct_cause_gestion_trafic_mois_n-3', 'prct_cause_gestion_trafic_mois_n-4',
                'prct_cause_materiel_roulant_passe','prct_cause_materiel_roulant_mois_n-1', 'prct_cause_materiel_roulant_mois_n-2', 'prct_cause_materiel_roulant_mois_n-3', 'prct_cause_materiel_roulant_mois_n-4',
                 'prct_cause_gestion_gare_passe','prct_cause_gestion_gare_mois_n-1', 'prct_cause_gestion_gare_mois_n-2', 'prct_cause_gestion_gare_mois_n-3', 'prct_cause_gestion_gare_mois_n-4',
                  'prct_cause_prise_en_charge_voyageurs_passe','prct_cause_prise_en_charge_voyageurs_mois_n-1', 'prct_cause_prise_en_charge_voyageurs_mois_n-2', 'prct_cause_prise_en_charge_voyageurs_mois_n-3', 'prct_cause_prise_en_charge_voyageurs_mois_n-4']]
     y_train = a[a['annee'] <= 2022][[column ]].values.ravel()
     y_test = a[a['annee'] == 2023][[column ]].values.ravel()

     #On commence par le one hot encoding des mois, des gares d'arrivée et de départ. 
     categorical_features = ["mois", "annee", "gare_depart", "gare_arrivee", "service", "commentaires_retard_arrivee"]
     #Les valeurs numériques 
     numeric_features = list(X.drop(["mois", "annee", "gare_depart", "gare_arrivee", "service", "commentaires_retard_arrivee"], axis = 1))


     pipeline_traitement_données = ColumnTransformer([
          ('std_scaler', StandardScaler(), numeric_features),
          ('one-hot', OneHotEncoder(), categorical_features)
     ])


     #On recrée un dataframe après la transformation pour pouvoir séparer train et test facilement

     X_transformed = pipeline_traitement_données.fit_transform(X)

     X_train_array = X_transformed.toarray()

     column_names_train = pipeline_traitement_données.get_feature_names_out(input_features=X.columns)


     # Recreate a DataFrame with column names and values
     X_final = pd.DataFrame(X_train_array, columns=column_names_train)

     X_test = X_final[X_final['one-hot__annee_2023']== 1 ]
     X_train = X_final.drop(X_final[X_final['one-hot__annee_2023']== 1].index)

     return [X_train, X_test, y_train, y_test]



In [ ]:


output = transfo_colonnes(df, 'retard_moyen_arrivee') #Ou retard_moyen_tous_trains_arrivee en fonction du cas considéré. 

X_train = output[0]
X_test = output[1]
y_train = output[2]
y_test = output[3]

On teste dans la cellule suivante différents modèles de regression et sortons leurs résultats après cross-validation sur notre training set.

In [ ]:
# Définir une liste de modèles de régression à tester
models = [
    ('Decision Tree', DecisionTreeRegressor()),
    ('Random Forest', RandomForestRegressor()),
    ('SVR', SVR()),
    ('K-Nearest Neighbors', KNeighborsRegressor()),
    ('XGbosst', HistGradientBoostingRegressor())
]


# Créer une matrice pour stocker les erreurs de chaque modèle
results = []

# Définissez une liste de métriques à utiliser
scoring = ['neg_mean_squared_error', 'r2']

# Effectuer la validation croisée pour chaque modèle
for i, (name, model) in enumerate(models):
    # Calculer les erreurs en utilisant la validation croisée
    scores = cross_validate(model, X_train, y_train, cv=3, scoring = scoring)
    
    # Calculer la racine carrée de l'erreur quadratique moyenne (RMSE)
    rmse_norm_scores = np.sqrt(-scores['test_neg_mean_squared_error'])
    
    
    # Stocker le nom du modèle et la moyenne des RMSE
    results.append((name, rmse_norm_scores.mean(), scores['test_r2'].mean()))


print(results)

Random forest à l'air d'être le modèle le plus efficace. Effectuons une grid search pour trouver les meilleurs paramètres. 

p.s. : Elle est très longue car on entraine de nombreux arbres de décision ce qui prend beaucoup de temps. Elle est donc commentée ici et les meilleurs hyper paramètres sont précisés dans la cellule suivante. 

In [ ]:
# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 

# rf_random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)],
#                'max_features': ['auto', 'sqrt'],
#                'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
#                'min_samples_split': [2, 5, 10],
#                'min_samples_leaf':  [1, 2, 4]}

# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = rf_random_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)

# grid_search.fit(X_train, y_train)

# # Obtenez les meilleurs hyperparamètres et le modèle optimal
# print(grid_search.best_params_)



In [ ]:
#On fait nos prédictions avec le meilleur modèle qui a les hyper paramètres suivants : 'max_depth': 60,'max_features': 'auto','min_samples_leaf': 4,'min_samples_split': 10,'n_estimators': 1000

rf = RandomForestRegressor(max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=1000)

rf.fit(X_train,y_train)


# Faire des prédictions sur l'ensemble de test
y_pred = rf.predict(X_test)

# Calculer le RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(rmse)


In [ ]:
#On compare la valeur retournée par notre modèle à la moyenne du retard 

print(y_test.mean(), np.sqrt(y_test.var()))
print((y_test.max(), y_test.min()))

## Cas 1

On obtient une rmse de 2,4 minutes. Cela parait plutot satisfaisant en particulier étant donné le peu d'informations contractuelles sur les trajets. 



## Cas 2

On obtient une rmse de 9,54 minutes. On remarque que nos prédictions ne sont pas particulièrement meilleures que la variance du dataset de test (13 minutes) et que nous sommes précis à environ un quart de la moyenne des valeurs de retards. Cela n'est pas extrèment convaincant. 
On peut mitiger cela car la valeur prédite par notre modèle ne sera probablement qu'une bonne première approximation pour la SNCF mais ne sera probablement pas meilleure que celle donnée par des informations en temps réel une fois que le train est en route. Une combinaison des deux approches pourrait probablement fournir les meilleurs résultats.


Dans le cas 2, nos prédictions sont moins bonnes que dans le cas 1. Cela vient probablement du fait que les trains qui ne sont pas en retard (retard = 0) viennent lisser les valeurs extrêmes. C'est ce qu'on observe pour Tourcoing_Bordeaux en mars 2023. On a retard_moyen_arrivee = 299.3 et retard_moyen_tous_trains_arrivee = 11.61. S'il y a un seul train en retard mais que celui ci est très en retard, on obtient une valeur énorme pour retard_moyen alors que retard_moyen_tous_trains_arrivee est lissé par tous les autres trains qui ne sont pas en retard. 